In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('ggplot')
%matplotlib inline

x_train = pd.read_csv('data/x_train_norm.csv', sep=';', na_values='?')
x_test = pd.read_csv('data/x_test_norm.csv', sep=';', na_values='?')
y_train = pd.read_csv('data/y_train.csv', sep=';', header=None, na_values='?')[0]

from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV

import scipy as sp
def logloss(act, pred):
    epsilon = 1e-15
    pred = sp.maximum(epsilon, pred)
    pred = sp.minimum(1-epsilon, pred)
    ll = sum(act*sp.log(pred) + sp.subtract(1,act)*sp.log(sp.subtract(1,pred)))
    ll = ll * -1.0/len(act)
    return ll

def scorer_logloss(estimator, X, y):
    return logloss(y, estimator.predict_proba(X)[:,1])

def revert_scorer_logloss(estimator, X, y):
    return -1 * scorer_logloss(estimator, X, y)

In [142]:
# from sklearn.model_selection import KFold
# def cross_validation_logloss(x_train, y_train, classifier, n_splits=5):
#     loglosses = []

#     X = x_train
#     y = y_train
#     kf = KFold(n_splits=n_splits) #, shuffle=True

#     i=1
#     for train_index, test_index in kf.split(x_train):
#         print '{}/{}'.format(i, n_splits)
#         i=i+1
        
#         xx_train, xx_test = x_train.loc[train_index], x_train.loc[test_index]
#         yy_train, yy_test = y[train_index], y[test_index]

#         classifier.fit(xx_train, yy_train)
#         prediction_proba = classifier.predict_proba(xx_test)[:,1]
#         ll = logloss(yy_test, prediction_proba)
    
#         loglosses.append(ll)
    
#     print loglosses, np.mean(loglosses)
#     return np.mean(loglosses)

# Использование
# classifier = KNeighborsClassifier(n_neighbors=50)
# cross_validation_logloss(x_train, y_train, classifier)

In [5]:
from sklearn.model_selection import cross_val_score

In [16]:
classifier = KNeighborsClassifier(n_neighbors=50)
cross_val_score(classifier, x_train, y_train, scoring=scorer_logloss, n_jobs=2, verbose=3)

[CV]  ................................................................
[CV]  ................................................................
[CV] ................................. , score=0.457319, total=   1.5s
[CV]  ................................................................
[CV] ................................. , score=0.442194, total=   1.5s
[CV] ................................. , score=0.460661, total=   1.1s


[Parallel(n_jobs=2)]: Done   3 out of   3 | elapsed:    2.7s finished


array([ 0.44219401,  0.4573191 ,  0.46066146])

In [17]:
classifier = SVC(probability=True)
cross_val_score(classifier, x_train, y_train, scoring=scorer_logloss, n_jobs=2, verbose=3)

[CV]  ................................................................
[CV]  ................................................................
[CV] ................................. , score=0.427338, total= 1.2min
[CV]  ................................................................
[CV] ................................. , score=0.423412, total= 1.2min
[CV] ................................. , score=0.426407, total=  58.0s


[Parallel(n_jobs=2)]: Done   3 out of   3 | elapsed:  2.2min finished


array([ 0.42341188,  0.42733813,  0.42640727])

дефолтный SVC лучше 50NN

Его и отправим

In [82]:
classifier = SVC(probability=True)
classifier.fit(x_train, y_train)
y_test = knn.predict_proba(x_test)[:,1]  # возвращается думерный массив, нас интересует 2-й стоблец
res_df = pd.DataFrame(y_test, columns = ['y'])
res_df.to_csv('res/003_svc.csv', sep=';', header=None, index=False)

In [ ]:
# c =10^3 считает очень долго. с=100 считает 15 минут
C_array = np.logspace(-3, 3, num=7) 
gamma_array = np.logspace(-5, 2, num=8)
res = []
minlogloss = np.Inf

i = 1
for C in C_array:
    print "C = {} ({}/{})".format(C,i,len(C_array))
    i=i+1
    j=1
    for gamma in gamma_array:
        print "gamms = {} ({}/{})".format(gamma,j,len(gamma_array))
        j=j+1
        
        classifier = SVC(probability=True, kernel='rbf', C=C, gamma=gamma)
        current_logloss = cross_validation_logloss(x_train, y_train, classifier)
        
        print current_logloss
        
        if current_logloss < minlogloss:
            minlogloss = current_logloss
            min_C = C
            min_gamma = gamma

        res.append((C, gamma, current_logloss))
     
print min_C, min_gamma, minlogloss


C = 0.001 (1/7)
gamms = 1e-05 (1/8)
1/5


Отправим случшими показателями

In [4]:
classifier = SVC(probability=True, C=10, gamma=0.001)
classifier.fit(x_train, y_train)
y_test = classifier.predict_proba(x_test)[:,1]  # возвращается думерный массив, нас интересует 2-й стоблец
res_df = pd.DataFrame(y_test, columns = ['y'])
res_df.to_csv('res/003_svc_C_10_gamma_0.001.csv', sep=';', header=None, index=False)

## Линейное ядро
kernel = linear

http://mlbootcamp.ru/article/tutorial/#Линейное-ядро

In [ ]:
C_array = np.logspace(-3, 3, num=7)
svc = SVC(probability=True, kernel='linear')
grid = GridSearchCV(svc, param_grid={'C': C_array}, scoring=revert_scorer_logloss, n_jobs=2, verbose=2)
grid.fit(x_train, y_train)

хороший показатель пори С <0.01. Там и поищем

In [ ]:
C_array = np.linspace(0.0001, 0.01, 10)
svc = SVC(probability=True, kernel='linear')
grid = GridSearchCV(svc, param_grid={'C': C_array}, scoring=scorer_logloss, n_jobs=2, verbose=3)
grid.fit(x_train, y_train)

In [ ]:
#результаты:
grid.grid_scores_

Отправить лучший результат

In [ ]:
classifier = SVC(probability=True, C=0.0012, kernel='linear')
classifier.fit(x_train, y_train)
y_test = classifier.predict_proba(x_test)[:,1]  # возвращается думерный массив, нас интересует 2-й стоблец
res_df = pd.DataFrame(y_test, columns = ['y'])
res_df.to_csv('res/003_svc_linear_C_0.0012.csv', sep=';', header=None, index=False)

# CV = 0.415547
# mlbootcamp = 0.4169297

результат хуже, чем дефолтный кернел

## Полиномиальное ядро
kernel = 'poly'

http://mlbootcamp.ru/article/tutorial/#Полиномиальное-ядро

In [19]:
C_array = np.logspace(-5, 2, num=8)
gamma_array = np.logspace(-5, 2, num=8)
degree_array = [2, 3, 4]
svc = SVC(probability=True, kernel='poly')
grid = GridSearchCV(svc, param_grid={'C': C_array, 'gamma': gamma_array, 'degree': degree_array}, n_jobs=3, verbose=3)
grid.fit(x_train, y_train)

Fitting 3 folds for each of 192 candidates, totalling 576 fits
[CV] C=1e-05, degree=2, gamma=1e-05 ..................................
[CV] C=1e-05, degree=2, gamma=1e-05 ..................................
[CV] C=1e-05, degree=2, gamma=1e-05 ..................................
[CV] C=1e-05, degree=2, gamma=0.0001 .................................
[CV] C=1e-05, degree=2, gamma=0.0001 .................................


KeyboardInterrupt: 